In [1]:
import pyspark
spark = pyspark.sql.SparkSession.builder.getOrCreate()
spark.range(5).show()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/19 11:28:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
+---+



In [2]:
import pyspark

spark = pyspark.sql.SparkSession.builder.getOrCreate()

In [3]:
import multiprocessing
import pyspark

nprocs = multiprocessing.cpu_count()

spark = (pyspark.sql.SparkSession.builder
 .master('local')
 .config('spark.jars.packages', 'mysql:mysql-connector-java:8.0.16')
 .config('spark.driver.memory', '4G')
 .config('spark.driver.cores', nprocs)
 .config('spark.sql.shuffle.partitions', nprocs)
 .appName('MySparkApplication')
 .getOrCreate())

23/05/19 11:28:09 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [4]:
import pyspark.sql.functions as F

In [5]:
import pandas as pd
import numpy as np

1. Create a spark data frame that contains your favorite programming languages.

- The name of the column should be language
- View the schema of the dataframe
- Output the shape of the dataframe
- Show the first 5 records in the dataframe

In [6]:
pro_lang_df = spark.createDataFrame(
    pd.DataFrame(
        {
            "language": [
                "MySQL",
                "Python",
                "R",
                "Java",
                "JavaScript",
                "Julia", 
                "HTML"
            ]
        }
    )
)

pro_lang_df.show(truncate=False)

+----------+
|language  |
+----------+
|MySQL     |
|Python    |
|R         |
|Java      |
|JavaScript|
|Julia     |
|HTML      |
+----------+



In [7]:
pro_lang_df.show()

+----------+
|  language|
+----------+
|     MySQL|
|    Python|
|         R|
|      Java|
|JavaScript|
|     Julia|
|      HTML|
+----------+



In [37]:
pro_lang_df.printSchema()

root
 |-- language: string (nullable = true)



In [8]:
pro_lang_df.count(), len(pro_lang_df.columns)

(7, 1)

In [9]:
pro_lang_df.show(5)

+----------+
|  language|
+----------+
|     MySQL|
|    Python|
|         R|
|      Java|
|JavaScript|
+----------+
only showing top 5 rows



2. Load the mpg dataset as a spark dataframe.
    
    Create 1 column of output that contains a message like the one below:

        -The 1999 audi a4 has a 4 cylinder engine.
        -For each vehicle.
        -Transform the trans column so that it only contains either manual or auto.

In [10]:
from pydataset import data
mpg = data('mpg')

mpg = spark.createDataFrame(mpg)

In [11]:
mpg.columns

['manufacturer',
 'model',
 'displ',
 'year',
 'cyl',
 'trans',
 'drv',
 'cty',
 'hwy',
 'fl',
 'class']

In [12]:
mpg.show()

+------------+------------------+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|             model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+------------------+-----+----+---+----------+---+---+---+---+-------+
|        audi|                a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|                a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|                a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|                a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|                a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
|        audi|                a4|  2.8|1999|  6|manual(m5)|  f| 18| 26|  p|compact|
|        audi|                a4|  3.1|2008|  6|  auto(av)|  f| 18| 27|  p|compact|
|        audi|        a4 quattro|  1.8|1999|  4|manual(m5)|  4| 18| 26|  p|compact|
|        audi|        a4 quattro|  1.8|1999|  4|  auto(l5)|  4| 16| 25|  p|c

In [13]:
mpg.filter(mpg.model == 'a4').where(mpg.year == 1999).show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
|        audi|   a4|  2.8|1999|  6|manual(m5)|  f| 18| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+



In [14]:
mpg.select(
    '*', 
    F.regexp_replace('trans', r'\(.*', '').alias('transmission'),
).show(truncate=False)

+------------+------------------+-----+----+---+----------+---+---+---+---+-------+------------+
|manufacturer|model             |displ|year|cyl|trans     |drv|cty|hwy|fl |class  |transmission|
+------------+------------------+-----+----+---+----------+---+---+---+---+-------+------------+
|audi        |a4                |1.8  |1999|4  |auto(l5)  |f  |18 |29 |p  |compact|auto        |
|audi        |a4                |1.8  |1999|4  |manual(m5)|f  |21 |29 |p  |compact|manual      |
|audi        |a4                |2.0  |2008|4  |manual(m6)|f  |20 |31 |p  |compact|manual      |
|audi        |a4                |2.0  |2008|4  |auto(av)  |f  |21 |30 |p  |compact|auto        |
|audi        |a4                |2.8  |1999|6  |auto(l5)  |f  |16 |26 |p  |compact|auto        |
|audi        |a4                |2.8  |1999|6  |manual(m5)|f  |18 |26 |p  |compact|manual      |
|audi        |a4                |3.1  |2008|6  |auto(av)  |f  |18 |27 |p  |compact|auto        |
|audi        |a4 quattro      

In [41]:
mpg.select(
    F.concat(F.lit('The '), F.expr('year'), 
                   F.lit(' '), F.expr('manufacturer'), F.lit(' '),
                  F.expr('model'), F.lit(' has a '), F.expr('cyl'),
                  F.lit( ' cylinder engine.')).alias('message')

).show(truncate=False)

+--------------------------------------------------------------+
|message                                                       |
+--------------------------------------------------------------+
|The 1999 audi a4 has a 4 cylinder engine.                     |
|The 1999 audi a4 has a 4 cylinder engine.                     |
|The 2008 audi a4 has a 4 cylinder engine.                     |
|The 2008 audi a4 has a 4 cylinder engine.                     |
|The 1999 audi a4 has a 6 cylinder engine.                     |
|The 1999 audi a4 has a 6 cylinder engine.                     |
|The 2008 audi a4 has a 6 cylinder engine.                     |
|The 1999 audi a4 quattro has a 4 cylinder engine.             |
|The 1999 audi a4 quattro has a 4 cylinder engine.             |
|The 2008 audi a4 quattro has a 4 cylinder engine.             |
|The 2008 audi a4 quattro has a 4 cylinder engine.             |
|The 1999 audi a4 quattro has a 6 cylinder engine.             |
|The 1999 audi a4 quattro

3. Load the tips dataset as a spark dataframe.

- What percentage of observations are smokers?
- Create a column that contains the tip percentage
- Calculate the average tip percentage for each combination of sex and smoker.

In [16]:
tips = data('tips')

tips = spark.createDataFrame(tips)

In [17]:
tips.show()

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|
|     18.43| 3.0|  Male|    No|Sun|Dinner|   4|
|     14.83|3.02|Female|    No|Sun|Dinner|   2|
|     21.58|3.92|  Male|    No|Sun|Dinner|   2|
|     10.33|1.67|Female|    No|Sun|Dinner|   3|
|     16.29|3.71|  Male|    No|Sun|Dinne

In [18]:
round((tips.where(tips.smoker == 'Yes').count() / tips.count()) *100, 2)

38.11

In [44]:
tips = tips.withColumn('tip_percent', ((tips.tip / tips.total_bill) * 100))

In [20]:
tips.show()

+----------+----+------+------+---+------+----+------------------+
|total_bill| tip|   sex|smoker|day|  time|size|       tip_percent|
+----------+----+------+------+---+------+----+------------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|5.9446733372572105|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|16.054158607350097|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|16.658733936220845|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2| 13.97804054054054|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|14.680764538430255|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4| 18.62396204033215|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2| 22.80501710376283|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|11.607142857142858|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|13.031914893617023|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|21.853856562922868|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2| 16.65043816942551|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|14.180374361883

In [21]:
tips.groupBy('sex').pivot('smoker').mean('tip_percent').show()

+------+-----------------+------------------+
|   sex|               No|               Yes|
+------+-----------------+------------------+
|Female|15.69209707691836| 18.21503526994103|
|  Male|16.06687151291298|15.277117520248513|
+------+-----------------+------------------+



4. Use the seattle weather dataset referenced in the lesson to answer the questions below.

    - Convert the temperatures to fahrenheit.
    - Which month has the most rain, on average?
    - Which year was the windiest?
    - What is the most frequent type of weather in January?
    - What is the average high and low temperature on sunny days in July in 2013 and 2014?
    - What percentage of days were rainy in q3 of 2015?
    - For each year, find what percentage of days it rained (had non-zero precipitation).

In [22]:
from vega_datasets import data

weather = data.seattle_weather().assign(date=lambda df: df.date.astype(str))
weather = spark.createDataFrame(weather)

In [23]:
weather.show()

+----------+-------------+--------+--------+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|
+----------+-------------+--------+--------+----+-------+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|
|2012-01-06|          2.5|     4.4|     2.2| 2.2|   rain|
|2012-01-07|          0.0|     7.2|     2.8| 2.3|   rain|
|2012-01-08|          0.0|    10.0|     2.8| 2.0|    sun|
|2012-01-09|          4.3|     9.4|     5.0| 3.4|   rain|
|2012-01-10|          1.0|     6.1|     0.6| 3.4|   rain|
|2012-01-11|          0.0|     6.1|    -1.1| 5.1|    sun|
|2012-01-12|          0.0|     6.1|    -1.7| 1.9|    sun|
|2012-01-13|          0.0|     5.0|    -2.8| 1.3|    sun|
|2012-01-14|          4.1|     4.4|     0.6| 5.3|   snow|
|2012-01-15|  

In [24]:
weather = weather.select(
    '*',
    F.expr('temp_max + 32 AS max_temp_f'),
    F.expr('temp_min + 32 AS min_temp_f')
)

In [26]:
weather.show()

+----------+-------------+--------+--------+----+-------+----------+----------+
|      date|precipitation|temp_max|temp_min|wind|weather|max_temp_f|min_temp_f|
+----------+-------------+--------+--------+----+-------+----------+----------+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|      44.8|      37.0|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|      42.6|      34.8|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|      43.7|      39.2|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|      44.2|      37.6|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|      40.9|      34.8|
|2012-01-06|          2.5|     4.4|     2.2| 2.2|   rain|      36.4|      34.2|
|2012-01-07|          0.0|     7.2|     2.8| 2.3|   rain|      39.2|      34.8|
|2012-01-08|          0.0|    10.0|     2.8| 2.0|    sun|      42.0|      34.8|
|2012-01-09|          4.3|     9.4|     5.0| 3.4|   rain|      41.4|      37.0|
|2012-01-10|          1.0|     6.1|     

In [36]:
weather.withColumn('month', F.month(F.col('date'))).groupby(
    'month').agg(
    F.round(F.mean(weather.precipitation), 2).alias('avg_rain')).show()

+-----+--------+
|month|avg_rain|
+-----+--------+
|    6|    1.11|
|    5|    1.67|
|    1|    3.76|
|    3|    4.89|
|    2|    3.73|
|    4|    3.13|
|   12|    5.02|
|    9|    1.96|
|   10|    4.06|
|    7|    0.39|
|   11|    5.35|
|    8|    1.32|
+-----+--------+



In [ ]:
# November on average has the most rain

In [45]:
weather.groupBy('weather').sum('precipitation').show()

+-------+------------------+
|weather|sum(precipitation)|
+-------+------------------+
|drizzle|               1.0|
|   snow|             208.1|
|   rain|1321.8000000000006|
|    sun|239.39999999999995|
|    fog|2655.7000000000003|
+-------+------------------+



In [50]:
weather.withColumn('month', F.month(F.col('date'))).groupBy('month').agg(F.mean('precipitation')).sort('avg(precipitation)').show()

+-----+-------------------+
|month| avg(precipitation)|
+-----+-------------------+
|    7|0.38870967741935486|
|    6| 1.1075000000000002|
|    8| 1.3201612903225806|
|    5| 1.6733870967741935|
|    9| 1.9624999999999997|
|    4|  3.128333333333333|
|    2|  3.734513274336283|
|    1| 3.7580645161290316|
|   10|  4.059677419354839|
|    3|  4.888709677419355|
|   12|  5.021774193548389|
|   11|  5.354166666666667|
+-----+-------------------+



In [51]:
weather.withColumn('year', F.year(F.col('date'))).groupby(
    'year').agg(
    F.round(F.mean(weather.wind), 2).alias('avg_wind')).show()

+----+--------+
|year|avg_wind|
+----+--------+
|2012|     3.4|
|2013|    3.02|
|2014|    3.39|
|2015|    3.16|
+----+--------+



In [55]:
weather.filter(
    F.month(weather.date) == 1
).groupBy('weather').count().sort(F.col('count').desc()).show()

+-------+-----+
|weather|count|
+-------+-----+
|    fog|   38|
|   rain|   35|
|    sun|   33|
|drizzle|   10|
|   snow|    8|
+-------+-----+



In [ ]:
# looks like the most frquent weather in January is fog

In [63]:
weather.filter(
    (F.month(weather.date) == 7) &
    (F.year(weather.date) > 2013) &
    (F.year(weather.date) < 2015) &
    (weather.weather == 'sun')
).agg(
    F.round(F.mean('max_temp_f'), 2), F.round(F.mean('min_temp_f'), 2)
).show()

+-------------------------+-------------------------+
|round(avg(max_temp_f), 2)|round(avg(min_temp_f), 2)|
+-------------------------+-------------------------+
|                    59.09|                     46.4|
+-------------------------+-------------------------+



In [72]:
weather.filter(
    F.quarter(weather.date) == 3).filter(
    F.year(weather.date) == 2015
).select(
    (F.when(weather.weather == 'rain', 1).otherwise(0)).alias('rain')
).agg((F.round(F.mean('rain') * 100, 2)).alias('percent_rain')).show()


+------------+
|percent_rain|
+------------+
|        2.17|
+------------+



In [78]:
weather.withColumn(
    'year',
    F.year(weather.date)).withColumn(
    'has_rained', 
    F.expr('precipitation > 0').cast('int')
).groupBy('year').agg(
    (F.round(F.avg('has_rained'), 4)).alias('percent_precip')).show()


+----+--------------+
|year|percent_precip|
+----+--------------+
|2012|        0.4836|
|2013|        0.4164|
|2014|         0.411|
|2015|        0.3945|
+----+--------------+

